El presente trabajo está basado en el análisis realizado por Hans Rolan Mersch. Contiene modificaciones según lo que se requiere para el presente proyecto.

In [503]:
import pandas as pd
import numpy as np

In [504]:
from platform import python_version
print(f"Python version: {python_version()}")
print(f"pandas version: {pd.__version__}")
print(f"numpy version: {np.__version__}")

Python version: 3.11.1
pandas version: 1.5.3
numpy version: 1.24.2


In [505]:
pd.set_option('display.max_columns', None)

# Carga de datos
El archivo de datos proveído del historial de la Facultad de Ingeniería es *datosConcatenados.csv*. Se procede a manipularlo utilizando `DataFrame` de pandas para tener una mejor organización y acceso.

In [506]:
df = pd.read_csv("data/datosConcatenados.csv")
print(df.shape)
df.head()

(221822, 23)


,Unnamed: 0,danho,ciclo,Cod.Asign,Asignatura,Cod.Car.Sec,Cod.Curso,Convocatoria,Anho,Semestre,Aprobado,Anho.Firma,Primer.Par,Segundo.Par,AOT,Primer.Rec,Segundo.Rec,Nota.Final,Tercer.Par,Asis,Cuarto.Par,4P_60,id_anony
0,0,2012,1,2402,COMBUSTION,INT9MECANI,13,1,2012,1,N,0,0,0,0,0,0,NaN,0.0,1.0,NaN,NaN,es_0
1,1,2012,1,2402,COMBUSTION,MEC9-OPT,13,1,2012,1,S,2012,11,14,8,0,0,1F-5,0.0,1.0,NaN,NaN,es_1
2,2,2012,1,2402,COMBUSTION,INT9MECANI,13,1,2012,1,S,2012,24,23,9,0,0,1F-5,0.0,1.0,NaN,NaN,es_2
3,3,2012,1,2402,COMBUSTION,MEC9-OPT,13,1,2012,1,S,2012,11,14,8,0,0,"1F-1,2F-2",0.0,1.0,NaN,NaN,es_3
4,4,2012,1,2402,COMBUSTION,MEC9-OPT,13,1,2012,1,N,0,11,4,8,0,0,NaN,0.0,1.0,NaN,NaN,es_4


# Eliminación de datos duplicados y no útiles

Las columnas `danho` y `Anho` son iguales, por lo que se elimina la primera de ellas:

In [507]:
print(df["danho"].equals(df["Anho"]))
df.drop(columns=["danho"], inplace=True)
df.head()

True


,Unnamed: 0,ciclo,Cod.Asign,Asignatura,Cod.Car.Sec,Cod.Curso,Convocatoria,Anho,Semestre,Aprobado,Anho.Firma,Primer.Par,Segundo.Par,AOT,Primer.Rec,Segundo.Rec,Nota.Final,Tercer.Par,Asis,Cuarto.Par,4P_60,id_anony
0,0,1,2402,COMBUSTION,INT9MECANI,13,1,2012,1,N,0,0,0,0,0,0,NaN,0.0,1.0,NaN,NaN,es_0
1,1,1,2402,COMBUSTION,MEC9-OPT,13,1,2012,1,S,2012,11,14,8,0,0,1F-5,0.0,1.0,NaN,NaN,es_1
2,2,1,2402,COMBUSTION,INT9MECANI,13,1,2012,1,S,2012,24,23,9,0,0,1F-5,0.0,1.0,NaN,NaN,es_2
3,3,1,2402,COMBUSTION,MEC9-OPT,13,1,2012,1,S,2012,11,14,8,0,0,"1F-1,2F-2",0.0,1.0,NaN,NaN,es_3
4,4,1,2402,COMBUSTION,MEC9-OPT,13,1,2012,1,N,0,11,4,8,0,0,NaN,0.0,1.0,NaN,NaN,es_4


Las columnas `Unnamed: 0` y `Cod.Curso` no aportan información relevante, ya que representan el número de registro en la tabla y el código interno utilizado por la facultad para cada materia, respectivamente. Por ello, se eliminan del dataframe:

In [508]:
df.drop(columns=["Unnamed: 0"], inplace=True)
df.drop(columns=["Cod.Curso"], inplace=True)
df.head()

,ciclo,Cod.Asign,Asignatura,Cod.Car.Sec,Convocatoria,Anho,Semestre,Aprobado,Anho.Firma,Primer.Par,Segundo.Par,AOT,Primer.Rec,Segundo.Rec,Nota.Final,Tercer.Par,Asis,Cuarto.Par,4P_60,id_anony
0,1,2402,COMBUSTION,INT9MECANI,1,2012,1,N,0,0,0,0,0,0,NaN,0.0,1.0,NaN,NaN,es_0
1,1,2402,COMBUSTION,MEC9-OPT,1,2012,1,S,2012,11,14,8,0,0,1F-5,0.0,1.0,NaN,NaN,es_1
2,1,2402,COMBUSTION,INT9MECANI,1,2012,1,S,2012,24,23,9,0,0,1F-5,0.0,1.0,NaN,NaN,es_2
3,1,2402,COMBUSTION,MEC9-OPT,1,2012,1,S,2012,11,14,8,0,0,"1F-1,2F-2",0.0,1.0,NaN,NaN,es_3
4,1,2402,COMBUSTION,MEC9-OPT,1,2012,1,N,0,11,4,8,0,0,NaN,0.0,1.0,NaN,NaN,es_4


Considerando que el año 2020 estuvo marcado por la pandemia de Covid-19, y el sistema de evaluación fue modificado temporalmente por ese evento (se añadieron un tercer parcial `Tercer.Par`, un cuarto parcial `Cuarto.Par` y un caso especial de sobreescritura de puntajes de parciales anteriores si en porcentaje el cuarto parcial era mejor que la suma de los otros 3, `4P_60`), considerar los resultados de ese año podría generar un sesgo en el modelo, por lo que se prefiere eliminar todos los registros de ese año y esas columnas:

In [509]:
print(df.loc[df["Tercer.Par"].notna() & df["Tercer.Par"] != 0.0, "Anho"].unique())
print(df.loc[df["Cuarto.Par"].notna(), "Anho"].unique())
print(df.loc[df["4P_60"].notna(), "Anho"].unique())

[2020]
[2020]
[2020]


In [510]:
df.drop(columns=["Tercer.Par"], inplace=True)
df.drop(columns=["Cuarto.Par"], inplace=True)
df.drop(columns=["4P_60"], inplace=True)

df = df[df["Anho"] != 2020]

In [511]:
df.head()

,ciclo,Cod.Asign,Asignatura,Cod.Car.Sec,Convocatoria,Anho,Semestre,Aprobado,Anho.Firma,Primer.Par,Segundo.Par,AOT,Primer.Rec,Segundo.Rec,Nota.Final,Asis,id_anony
0,1,2402,COMBUSTION,INT9MECANI,1,2012,1,N,0,0,0,0,0,0,NaN,1.0,es_0
1,1,2402,COMBUSTION,MEC9-OPT,1,2012,1,S,2012,11,14,8,0,0,1F-5,1.0,es_1
2,1,2402,COMBUSTION,INT9MECANI,1,2012,1,S,2012,24,23,9,0,0,1F-5,1.0,es_2
3,1,2402,COMBUSTION,MEC9-OPT,1,2012,1,S,2012,11,14,8,0,0,"1F-1,2F-2",1.0,es_3
4,1,2402,COMBUSTION,MEC9-OPT,1,2012,1,N,0,11,4,8,0,0,NaN,1.0,es_4


# Relleno de datos no válidos
Se verifica si existen valores `NaN`:

In [512]:
df.isna().sum()

ciclo               0
Cod.Asign           0
Asignatura          0
Cod.Car.Sec         0
Convocatoria        0
Anho                0
Semestre            0
Aprobado            0
Anho.Firma          0
Primer.Par          0
Segundo.Par         0
AOT                 0
Primer.Rec          0
Segundo.Rec         0
Nota.Final      73819
Asis                0
id_anony            0
dtype: int64

Todas las ocurrencias son en `Nota.Final`, se rellena con 0, y dado el formato particular que tienen los valores de esa columna, se sabe que si el valor es 0, el alumno no rindió ningún final:

In [513]:
df["Nota.Final"].fillna(0, inplace=True)
df.head()

,ciclo,Cod.Asign,Asignatura,Cod.Car.Sec,Convocatoria,Anho,Semestre,Aprobado,Anho.Firma,Primer.Par,Segundo.Par,AOT,Primer.Rec,Segundo.Rec,Nota.Final,Asis,id_anony
0,1,2402,COMBUSTION,INT9MECANI,1,2012,1,N,0,0,0,0,0,0,0,1.0,es_0
1,1,2402,COMBUSTION,MEC9-OPT,1,2012,1,S,2012,11,14,8,0,0,1F-5,1.0,es_1
2,1,2402,COMBUSTION,INT9MECANI,1,2012,1,S,2012,24,23,9,0,0,1F-5,1.0,es_2
3,1,2402,COMBUSTION,MEC9-OPT,1,2012,1,S,2012,11,14,8,0,0,"1F-1,2F-2",1.0,es_3
4,1,2402,COMBUSTION,MEC9-OPT,1,2012,1,N,0,11,4,8,0,0,0,1.0,es_4


# Preprocesamiento
Observando las columnas `Aprobado` y `Asistencia`:

In [514]:
print(df["Aprobado"].unique())
print(df["Asis"].unique())

['N' 'S']
[ 1.  0. 11.]


`Aprobado` contiene `S` si el alumno aprobó y `N` si no aprobó. `Asis` contiene `1.` (en algunos casos erróneamente `11.`) y `0.` según el alumno cumplió con los requerimientos de asistencia para estar habilitado a rendir el examen final.

Se reemplazan por valores booleanos:

In [515]:
df["Aprobado"].replace({"S": True, "N": False}, inplace=True)
df["Asis"].replace({1.: True, 11.: True ,0.: False}, inplace=True)
df.head()

,ciclo,Cod.Asign,Asignatura,Cod.Car.Sec,Convocatoria,Anho,Semestre,Aprobado,Anho.Firma,Primer.Par,Segundo.Par,AOT,Primer.Rec,Segundo.Rec,Nota.Final,Asis,id_anony
0,1,2402,COMBUSTION,INT9MECANI,1,2012,1,False,0,0,0,0,0,0,0,True,es_0
1,1,2402,COMBUSTION,MEC9-OPT,1,2012,1,True,2012,11,14,8,0,0,1F-5,True,es_1
2,1,2402,COMBUSTION,INT9MECANI,1,2012,1,True,2012,24,23,9,0,0,1F-5,True,es_2
3,1,2402,COMBUSTION,MEC9-OPT,1,2012,1,True,2012,11,14,8,0,0,"1F-1,2F-2",True,es_3
4,1,2402,COMBUSTION,MEC9-OPT,1,2012,1,False,0,11,4,8,0,0,0,True,es_4


Se renombran las columnas para un mejor entendimiento:

In [516]:
df.rename(columns={
    'ciclo' : 'Ciclo',
    'Cod.Asign':'CodigoAsignatura',
    'Cod.Car.Sec':'CodigoCarrera',
    'Primer.Par':'1P',
    'Segundo.Par':'2P',
    'AOT':'Taller',
    'Anho.Firma':'AnhoFirma',
    'Primer.Rec':'1R',
    'Segundo.Rec':'2R',
    'Nota.Final':'NotaFinal',
    'Asis':'Asistencia',
    'id_anony':'IdAnonimizado'
    
    }, inplace=True)

df.head()

,Ciclo,CodigoAsignatura,Asignatura,CodigoCarrera,Convocatoria,Anho,Semestre,Aprobado,AnhoFirma,1P,2P,Taller,1R,2R,NotaFinal,Asistencia,IdAnonimizado
0,1,2402,COMBUSTION,INT9MECANI,1,2012,1,False,0,0,0,0,0,0,0,True,es_0
1,1,2402,COMBUSTION,MEC9-OPT,1,2012,1,True,2012,11,14,8,0,0,1F-5,True,es_1
2,1,2402,COMBUSTION,INT9MECANI,1,2012,1,True,2012,24,23,9,0,0,1F-5,True,es_2
3,1,2402,COMBUSTION,MEC9-OPT,1,2012,1,True,2012,11,14,8,0,0,"1F-1,2F-2",True,es_3
4,1,2402,COMBUSTION,MEC9-OPT,1,2012,1,False,0,11,4,8,0,0,0,True,es_4


Se puede saber si en una determinada cursada, el alumno ya tiene derecho a examen final con la información proveída por la columna `AnhoFirma`. Para acceder más fácilmente a esa información se crea una columna `ConFirmaPrevia` y luego ya no es necesaria la columna `AnhoFirma`:

In [517]:
con_firma_previa = df["AnhoFirma"] != 0
df.insert(7, "ConFirmaPrevia", con_firma_previa, True)
df.drop(columns=["AnhoFirma"], inplace=True)
df.head()

,Ciclo,CodigoAsignatura,Asignatura,CodigoCarrera,Convocatoria,Anho,Semestre,ConFirmaPrevia,Aprobado,1P,2P,Taller,1R,2R,NotaFinal,Asistencia,IdAnonimizado
0,1,2402,COMBUSTION,INT9MECANI,1,2012,1,False,False,0,0,0,0,0,0,True,es_0
1,1,2402,COMBUSTION,MEC9-OPT,1,2012,1,True,True,11,14,8,0,0,1F-5,True,es_1
2,1,2402,COMBUSTION,INT9MECANI,1,2012,1,True,True,24,23,9,0,0,1F-5,True,es_2
3,1,2402,COMBUSTION,MEC9-OPT,1,2012,1,True,True,11,14,8,0,0,"1F-1,2F-2",True,es_3
4,1,2402,COMBUSTION,MEC9-OPT,1,2012,1,False,False,11,4,8,0,0,0,True,es_4


Las notas finales vienen agrupadas en un string. Conviene separar las notas de los 3 finales en columnas separadas. Para ello es necesario conocer su formato:

In [518]:
df["NotaFinal"].unique()

array([0, '1F-5', '1F-1,2F-2', '1F-4', '1F-3', '2F-4', '2F-2', '2F-5',
       '1F-1,2F-3', '1F-1,2F-1', '2F-1', '1F-1,2F-4', '2F-3', '1F-2',
       '3F-3', '3F-2', '1F-1,3F-3', '3F-4', '3F-5', '2F-1,3F-3',
       '2F-1,3F-2', '2F-1,3F-5', '1F-1,2F-1,3F-4', '1F-1', '2F-1,3F-4',
       '1F-1,3F-5', '2F-3,1F-1', '2F-1,3F-1', '1F-1,3F-2',
       '1F-1,2F-1,3F-3', '3F-1', '1F-1,2F-1,3F-1', '1F-1,3F-4', '1F-5F',
       '1F-1,3F-1', '2F-4,1F-1', '1F-1,2F-1,3F-2', '2F-2,1F-1',
       '1F-1,2F-5', '1F-1,2F-1,3F-5', '2F-1,1F-1,3F-5', '3F-5,1F-1',
       '3F-2,2F-1', '3F-1,2F-1', '1F-1,3F-5,2F-1', '3F-3,1F-1',
       '3F-4,2F-1', '2F-1,1F-1,3F-1', '3F-5,2F-1', '2F-1,1F-1,3F-3',
       '2F-1,1F-1', '2F-1,1F-1,3F-4', '2F-5F', '1F-1,3F-1,2F-1',
       '3F-3,2F-1', '2F-1,1F-1,3F-2', '2F-5,1F-1', '3F-2,1F-1',
       '3F-4,1F-1', '3F-1,1F-1', '3F-1,1F-1,2F-1', '1F-1,3F-2,2F-1',
       '3F-2,2F-1,1F-1', '3F-3,2F-1,1F-1', '3F-5F', '1F-1,3F-4,2F-1',
       '3F-2,1F-1,2F-1', '1F-1,3F-3,2F-1', '3F-3,1F-1,2F

Todas las notas finales terminan con `-<N>` donde `<N>` es la nota de ese examen en particular (entre 1 y 5), y los diferentes exámenes vienen separados por comas (`,`).

In [519]:
notas_finales_agrupadas = df["NotaFinal"]

In [520]:
notas_primer_final = []
notas_segundo_final = []
notas_tercer_final = []

for nota_agrupada in notas_finales_agrupadas:
    notas_primer_final.append(0)
    notas_segundo_final.append(0)
    notas_tercer_final.append(0)

    if nota_agrupada == 0:
        continue
    
    notas_separadas: list[str] = nota_agrupada.split(",")
    for indice, nota_examen in enumerate(notas_separadas):

        nota_examen_recortada = nota_examen.rstrip("F") # algunas calificaciones son 5 "Felicitado"
        valor_nota = int(nota_examen_recortada[-1])

        if "1F" in nota_examen_recortada:
            notas_primer_final[-1] = valor_nota
            continue
        
        if "2F" in nota_examen_recortada:
            notas_segundo_final[-1] = valor_nota
            continue
        
        if "3F" in nota_examen_recortada:
            notas_tercer_final[-1] = valor_nota
            continue

        if "ME" in nota_examen_recortada:
            if indice == 0 or indice == 1:
                notas_primer_final[-1] = valor_nota
            elif indice == 2:
                notas_segundo_final[-1] = valor_nota
            elif indice == 3:
                notas_tercer_final[-1] = valor_nota
        

In [521]:
df["1F"] = notas_primer_final
df["2F"] = notas_segundo_final
df["3F"] = notas_tercer_final

In [522]:
df.drop(columns=["NotaFinal", "Convocatoria"], inplace=True)
df.head()

,Ciclo,CodigoAsignatura,Asignatura,CodigoCarrera,Anho,Semestre,ConFirmaPrevia,Aprobado,1P,2P,Taller,1R,2R,Asistencia,IdAnonimizado,1F,2F,3F
0,1,2402,COMBUSTION,INT9MECANI,2012,1,False,False,0,0,0,0,0,True,es_0,0,0,0
1,1,2402,COMBUSTION,MEC9-OPT,2012,1,True,True,11,14,8,0,0,True,es_1,5,0,0
2,1,2402,COMBUSTION,INT9MECANI,2012,1,True,True,24,23,9,0,0,True,es_2,5,0,0
3,1,2402,COMBUSTION,MEC9-OPT,2012,1,True,True,11,14,8,0,0,True,es_3,1,2,0
4,1,2402,COMBUSTION,MEC9-OPT,2012,1,False,False,11,4,8,0,0,True,es_4,0,0,0


# Categorizaciones
## Categorización de Asignaturas

In [523]:
asignaturas = df["Asignatura"].unique()

df["Asignatura"] = pd.Categorical(df["Asignatura"])
df["Asignatura"] = df["Asignatura"].cat.codes

codigos_asignaturas = np.unique(df["Asignatura"])

identificador_asignaturas = pd.DataFrame(data=asignaturas, index=None, columns=["Asignatura"])
identificador_asignaturas["Codigo"] = codigos_asignaturas
identificador_asignaturas
identificador_asignaturas.to_csv("data/identificadorAsignaturas.csv", index=False)

## Categorización de Carreras

In [524]:
df["CodigoCarrera"] = df["CodigoCarrera"].str.slice(0,3)

carreras = df["CodigoCarrera"].unique()

df["CodigoCarrera"] = pd.Categorical(df["CodigoCarrera"])
df["CodigoCarrera"] = df["CodigoCarrera"].cat.codes

codigos_carreras = np.unique(df["CodigoCarrera"])

identificador_carreras = pd.DataFrame(data=carreras, index=None, columns=["Carrera"])
identificador_carreras["CodigoCarrera"] = codigos_carreras
identificador_carreras
identificador_carreras.to_csv("data/identificadorCarreras.csv", index=False)

In [525]:
df.head()

,Ciclo,CodigoAsignatura,Asignatura,CodigoCarrera,Anho,Semestre,ConFirmaPrevia,Aprobado,1P,2P,Taller,1R,2R,Asistencia,IdAnonimizado,1F,2F,3F
0,1,2402,49,5,2012,1,False,False,0,0,0,0,0,True,es_0,0,0,0
1,1,2402,49,7,2012,1,True,True,11,14,8,0,0,True,es_1,5,0,0
2,1,2402,49,5,2012,1,True,True,24,23,9,0,0,True,es_2,5,0,0
3,1,2402,49,7,2012,1,True,True,11,14,8,0,0,True,es_3,1,2,0
4,1,2402,49,7,2012,1,False,False,11,4,8,0,0,True,es_4,0,0,0


In [526]:
df["IdAnonimizado"] = df["IdAnonimizado"].str.replace("es_", "").astype(int)

# Dataset final

In [528]:
df.to_csv("data/DatasetFiltrado.csv", index=False)